In [1]:
from langchain_community.llms import Ollama
from langchain.memory import ConversationBufferMemory
import os
from dotenv import load_dotenv
import langchain
# from langchain.llms import LLaMA
from langchain.chains import ConversationChain
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

from langchain_community.llms import Ollama
import streamlit as st
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
loader=TextLoader('generated_conversations.txt')

In [3]:
loader

In [4]:
# To convert loader into text function/document function  we use load() function
text_documents=loader.load()

In [5]:
from langchain_text_splitters import CharacterTextSplitter
text_splitter=CharacterTextSplitter(separator="\n\n",chunk_size=260,chunk_overlap=20)
pks=text_splitter.split_documents(text_documents)

Created a chunk of size 261, which is longer than the specified 260
Created a chunk of size 261, which is longer than the specified 260
Created a chunk of size 264, which is longer than the specified 260
Created a chunk of size 264, which is longer than the specified 260
Created a chunk of size 261, which is longer than the specified 260
Created a chunk of size 262, which is longer than the specified 260
Created a chunk of size 262, which is longer than the specified 260
Created a chunk of size 264, which is longer than the specified 260
Created a chunk of size 263, which is longer than the specified 260
Created a chunk of size 262, which is longer than the specified 260
Created a chunk of size 263, which is longer than the specified 260
Created a chunk of size 263, which is longer than the specified 260
Created a chunk of size 264, which is longer than the specified 260
Created a chunk of size 261, which is longer than the specified 260
Created a chunk of size 263, which is longer tha

In [6]:
speech=""
with open("generated_conversations.txt") as f:
    speech=f.read()
text_splitter=CharacterTextSplitter(chunk_size=300,chunk_overlap=20)
text=text_splitter.create_documents([speech])

In [7]:
print(text[0])
#print(text[1])

page_content='User: I would like to apply for maternity leave
AIChatbot: which type of leave?
User: annual leave
AIChatbot: how many days and date?
User: 2 days from 10-OCT-2024
AIChatbot: Thanks for the information, you can follow the link https://sphinx.shrmpro.com/'


In [8]:

print(text[1])

page_content='User: I need to go on leave
AIChatbot: which type of leave?
User: bereavement leave
AIChatbot: how many days and date?
User: 4 days starting 15-NOV-2024
AIChatbot: Thanks for the information, you can follow the link https://sphinx.shrmpro.com/'


In [9]:
len(text)

1500

In [10]:
type(text)

list

In [11]:
type(text[0])

langchain_core.documents.base.Document

In [12]:
from langchain_community.embeddings import OllamaEmbeddings

In [13]:
embeddings=(
    OllamaEmbeddings(model="llama3.1")  ##by default it ues llama2
)

/var/folders/jp/52lg7vhx5xqc9l1fx_d3rk300000gn/T/ipykernel_27621/1904063905.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  OllamaEmbeddings(model="llama3.1")  ##by default it ues llama2


In [14]:
# vectorstore = create_embeddings_in_batches(text, embeddings)

In [15]:
# vectorstore=Chroma.from_documents(documents=pks,embedding=embeddings)
# retriever=vectorstore.as_retriever()
# retriever

In [16]:
from langchain_community.vectorstores import FAISS
embeddings=OllamaEmbeddings(model="llama3.1")
db=FAISS.from_documents(text[0:800],embeddings)
db

In [17]:
type(db)

langchain_community.vectorstores.faiss.FAISS

In [18]:
### querying 
query="whats your plan"
docs=db.similarity_search(query)
docs

[Document(metadata={}, page_content='User: I want to submit a leave request\nAIChatbot: which type of leave?\nUser: maternity leave\nAIChatbot: how many days and date?\nUser: 3 days from 15-OCT-2024\nAIChatbot: Thanks for the information, you can follow the link https://sphinx.shrmpro.com/'),
 Document(metadata={}, page_content='User: I want to submit a leave request\nAIChatbot: which type of leave?\nUser: maternity leave\nAIChatbot: how many days and date?\nUser: 3 days from 15-OCT-2024\nAIChatbot: Thanks for the information, you can follow the link https://sphinx.shrmpro.com/'),
 Document(metadata={}, page_content='User: I am planning to go on holiday\nAIChatbot: which type of leave?\nUser: study leave\nAIChatbot: how many days and date?\nUser: 1 day on 08-OCT-2024\nAIChatbot: Thanks for the information, you can follow the link https://sphinx.shrmpro.com/'),
 Document(metadata={}, page_content='User: I want to submit a leave request\nAIChatbot: which type of leave?\nUser: maternity l

In [19]:
docs[0].page_content

'User: I want to submit a leave request\nAIChatbot: which type of leave?\nUser: maternity leave\nAIChatbot: how many days and date?\nUser: 3 days from 15-OCT-2024\nAIChatbot: Thanks for the information, you can follow the link https://sphinx.shrmpro.com/'

In [20]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'User: I want to submit a leave request\nAIChatbot: which type of leave?\nUser: maternity leave\nAIChatbot: how many days and date?\nUser: 3 days from 15-OCT-2024\nAIChatbot: Thanks for the information, you can follow the link https://sphinx.shrmpro.com/'

In [21]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(metadata={}, page_content='User: I want to submit a leave request\nAIChatbot: which type of leave?\nUser: maternity leave\nAIChatbot: how many days and date?\nUser: 3 days from 15-OCT-2024\nAIChatbot: Thanks for the information, you can follow the link https://sphinx.shrmpro.com/'),
  30054.092),
 (Document(metadata={}, page_content='User: I want to submit a leave request\nAIChatbot: which type of leave?\nUser: maternity leave\nAIChatbot: how many days and date?\nUser: 3 days from 15-OCT-2024\nAIChatbot: Thanks for the information, you can follow the link https://sphinx.shrmpro.com/'),
  30054.092),
 (Document(metadata={}, page_content='User: I am planning to go on holiday\nAIChatbot: which type of leave?\nUser: study leave\nAIChatbot: how many days and date?\nUser: 1 day on 08-OCT-2024\nAIChatbot: Thanks for the information, you can follow the link https://sphinx.shrmpro.com/'),
  30076.574),
 (Document(metadata={}, page_content='User: I want to submit a leave request\nAICh

In [22]:
# from langchain.llms import LLaMA

In [23]:
llm_model=Ollama(model="llama3.1")

/var/folders/jp/52lg7vhx5xqc9l1fx_d3rk300000gn/T/ipykernel_27621/1086579521.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm_model=Ollama(model="llama3.1")


In [31]:
# # Define the prompt template
# from langchain.prompts import PromptTemplate
# from langchain.chains import ConversationChain
# from langchain.memory import ConversationBufferMemory
# prompt_template = PromptTemplate(
#     input_variables=["user_input"],  # Define any input variables
#     template="Respond as a leave bot assistant. User said: {user_input}"
# )

In [25]:
# memory = ConversationBufferMemory()

/var/folders/jp/52lg7vhx5xqc9l1fx_d3rk300000gn/T/ipykernel_27621/995385594.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


In [32]:
# conversation_chain = ConversationChain(
#     llm=llm_model,
#     prompt=prompt_template,
#     memory=memory  # Include the memory component
# )

In [36]:

def sphinx_chat_bot(vector_store):
    print("Welcome to sphinx_chat_bot!")
    
    context = {}

    while True:
        user_input = input("User: ")
        if user_input.lower() in ["exit", "quit"]:
            print("sphinx_chat_bot: Thank you for using thesphinx bot. Goodbye!")
            break 
        print(user_input)
        
        # Find relevant responses based on user input using similarity_search
        relevant_responses = vector_store.similarity_search(user_input)

        if relevant_responses:
            # Choose the best response means first rows
            response = relevant_responses[0].page_content  # Get the best match
            print("sphinx_chat_bot:", response)
            
            # Handle conversation flow manually 
            if "What type of leave would you like to apply for?" in response:
                leave_type = input("User: ")
                print(leave_type)
                context["leave_type"] = leave_type
                
            elif "How many days of leave would you like to take" in response:
                days_input = input("User: ")
                print("days input:",days_input)
                context["days"] = days_input
                
                # Additional prompt for final confirmation
                final_response = f"Thank you for providing the details! You've applied for {context.get('leave_type')} for {context.get('days')} days. You can submit your request at the following link: http://hrms.sphinworldbiz.com"
                print("sphinx_chat_bot:", final_response)
                break  # Exit after final response

            # If user asks for other information
            elif "How can I assist you today?" in response:
                print("sphinx_chat_bot: If you have any questions about your leave or need assistance, feel free to ask.")
        else:
            print("sphinx_chat_bot: I'm sorry, I couldn't find an appropriate response.")


In [2]:
sphinx_chat_bot(db)#days as input

Welcome to sphinx_chat_bot!

sphinx_chat_bot: User: I need to request a day off
AIChatbot: which type of leave?
User: personal leave
AIChatbot: how many days and date?
User: 1 day on 08-OCT-2024
AIChatbot: Thanks for the information, you can follow the link https://sphinx.shrmpro.com/
"I want a leave"
sphinx_chat_bot: User: I want to submit a leave request
AIChatbot: which type of leave?
User: holiday leave
AIChatbot: how many days and date?
User: 1 day on 01-NOV-2024
AIChatbot: Thanks for the information, you can follow the link https://sphinx.shrmpro.com/
"i want a day off"
sphinx_chat_bot: User: I am planning to go on holiday
AIChatbot: which type of leave?
User: sick leave
AIChatbot: how many days and date?
User: 4 days starting 20-OCT-2024
AIChatbot: Thanks for the information, you can follow the link https://sphinx.shrmpro.com/
4
sphinx_chat_bot: User: I want to submit a leave request
AIChatbot: which type of leave?
User: maternity leave
AIChatbot: how many days and date?
User: 2